In [1]:
from pprint import pprint

import os
import time
import json
import uuid

from nltk.tokenize import TreebankWordTokenizer

from representation.container import TemporalContainer, Ruler, TemporalRuler
from representation.entity import Friend, Person, Object, Gender, Emotion
from representation.util import Identifier, serializer
from representation.entity import Friend, Gender, Emotion
from representation.scenario import Scenario, ScenarioContext, Modality, Mention, Annotation
from representation.scenario import ImageSignal, TextSignal, append_signal
from representation.mention import Utterance, Token, Triple
from representation.util import serializer

Define a time segments for the signals.

In [2]:
def scenario_data_path(scenario_id, file_name, modality=None, data_dir="data/test-scenes"):
    path = os.path.join(data_dir, str(scenario_id))
    if modality:
        path = os.path.join(path, modality.name.lower())
    
    return os.path.join(path, file_name)

def clean(scenario_id, data_dir="data/test-scenes"):
    # clean data
    path = scenario_data_path(scenario_id, "", data_dir=data_dir)
    json_files = (os.path.join(root, file)
             for root, _, files in os.walk(path)
             for file in files if file.endswith(".json"))

    for file in json_files:
        try:
            os.remove(file)
            print("Cleaned", file)
        except OSError:
            pass


In [3]:
scenario_id = "test_scenario"
clean(scenario_id)

piek = Friend(None, "Piek", 59, Gender.MALE)
context = ScenarioContext("leolani", piek, [], [])
signals = {
    Modality.IMAGE.name.lower(): scenario_data_path(scenario_id, Modality.IMAGE.name.lower() + ".json"),
    Modality.TEXT.name.lower(): scenario_data_path(scenario_id, Modality.TEXT.name.lower() + ".json")
}
scenario = Scenario(scenario_id, 1603139000, 1603150000, context, signals)

print(json.dumps(scenario, default=serializer, indent=4))

scenario_path = scenario_data_path(scenario_id, str(scenario_id) + ".json")
os.makedirs(os.path.dirname(scenario_path), exist_ok=True)
with open(scenario_path, 'w') as scenario_file:
    json.dump(scenario, scenario_file, default=serializer, indent=4)


Cleaned data/test-scenes/test_scenario/test_scenario.json
Cleaned data/test-scenes/test_scenario/image.json
Cleaned data/test-scenes/test_scenario/text.json
{
    "start_time": 1603139000,
    "end_time": 1603150000,
    "id": "test_scenario",
    "ruler": {
        "type": "TemporalRuler",
        "container_id": "test_scenario",
        "start": 1603139000,
        "end": 1603150000
    },
    "context": {
        "agent": "leolani",
        "speaker": {
            "id": "4dece423-03e6-4817-8904-1fbc81e14425",
            "name": "Piek",
            "age": 59,
            "gender": "MALE"
        },
        "persons": [],
        "objects": []
    },
    "signals": {
        "image": "data/test-scenes/test_scenario/image.json",
        "text": "data/test-scenes/test_scenario/text.json"
    }
}


Define the speaker and his/her emotion.

In [4]:
speaker_img_file = scenario_data_path(scenario_id, "piek-1.jpg", Modality.IMAGE)
speaker_img_time = scenario.ruler.get_time_segment(1603139705, 1603140000)

speaker_img_signal = ImageSignal(None, speaker_img_time, [speaker_img_file], ((0, 550), (0, 550)))
speaker_bounding_box = speaker_img_signal.ruler.get_area_bounding_box(10,15,521,518)

piek = Friend(None, "Piek", 59, Gender.MALE)
speaker_annotation = Annotation(piek, "face_recognition", time.time())
emotion_annotation = Annotation(Emotion.JOY, "annotator_1", time.time())

speaker_mention = Mention(speaker_bounding_box, [speaker_annotation, emotion_annotation])
speaker_img_signal.mentions.append(speaker_mention)

print(json.dumps(speaker_img_signal, default=serializer, indent=4))
append_signal(scenario.signals[Modality.IMAGE.name.lower()], speaker_img_signal)

{
    "modality": "IMAGE",
    "time": {
        "type": "TemporalRuler",
        "container_id": "test_scenario",
        "start": 1603139705,
        "end": 1603140000
    },
    "files": [
        "data/test-scenes/test_scenario/image/piek-1.jpg"
    ],
    "mentions": [
        {
            "segment": {
                "type": "MultiIndex",
                "container_id": "f7c836b8-64b7-474c-afcd-46df744e1e6d",
                "bounds": [
                    [
                        10,
                        521
                    ],
                    [
                        15,
                        518
                    ]
                ]
            },
            "annotations": [
                {
                    "type": "Friend",
                    "value": {
                        "id": "dc74258c-2849-4c62-b6b8-e6df13d8e653",
                        "name": "Piek",
                        "age": 59,
                        "gender": "MALE"
                

In [5]:
def tokenize(text):
    offsets = tuple(TreebankWordTokenizer().span_tokenize(text))
    
    return tuple(((start, end), text[start:end]) for start, end in offsets)
    
pprint([t for t in enumerate(tokenize("That is my brother Jim"))])

[(0, ((0, 4), 'That')),
 (1, ((5, 7), 'is')),
 (2, ((8, 10), 'my')),
 (3, ((11, 18), 'brother')),
 (4, ((19, 22), 'Jim'))]


In [6]:
speaker = piek

transcript = scenario_data_path(scenario_id, "chat1_utterance1.txt", Modality.TEXT)
with open(transcript) as f:
    utterance = f.readline()

utterance_time = scenario.ruler.get_time_segment(1603139850, 1603149890)
text_signal =  TextSignal(None, utterance_time, [transcript], len(utterance))

tokenized = tokenize(utterance)
tokens = [Token(token[1]) for token in tokenized]
token_segments = [text_signal.ruler.get_offset(*token[0]) for token in tokenized]

token_mentions = [Mention(segment, [Annotation(token, "treebank_tokenizer", time.time())])
                  for token, segment in zip(tokens, token_segments)]
text_signal.mentions.extend(token_mentions)

print(json.dumps(token_mentions, default=serializer, indent=4))

[
    {
        "segment": {
            "type": "Index",
            "container_id": "2f011b41-28ed-4184-8e6c-fcdeac4c964b",
            "start": 0,
            "stop": 4
        },
        "annotations": [
            {
                "type": "Token",
                "value": {
                    "value": "That",
                    "id": "7bb5ca79-5b25-4864-a2c9-14f67d1c2287",
                    "ruler": {
                        "type": "AtomicRuler",
                        "container_id": "7bb5ca79-5b25-4864-a2c9-14f67d1c2287"
                    }
                },
                "source": "treebank_tokenizer",
                "timestamp": 1605002037.027455
            }
        ]
    },
    {
        "segment": {
            "type": "Index",
            "container_id": "2f011b41-28ed-4184-8e6c-fcdeac4c964b",
            "start": 5,
            "stop": 7
        },
        "annotations": [
            {
                "type": "Token",
                "value": {
           

In [7]:
annotation_time = time.time()
utterance_annotation = Annotation(Utterance(None, utterance, tokens), "annotator_1", annotation_time)
utterance_speaker = Annotation(piek, "annotator_1", annotation_time)
utterance_emotion = Annotation(Emotion.HAPPINESS, "annotator_1", annotation_time)
utterance_mention = Mention([t.ruler for t in tokens], [utterance_annotation, utterance_speaker, utterance_emotion])
text_signal.mentions.append(utterance_mention)

print(json.dumps(utterance_mention, default=serializer, indent=4))

{
    "segment": [
        {
            "type": "AtomicRuler",
            "container_id": "7bb5ca79-5b25-4864-a2c9-14f67d1c2287"
        },
        {
            "type": "AtomicRuler",
            "container_id": "a5a605c8-b99a-4f77-b8b4-75393a2140d8"
        },
        {
            "type": "AtomicRuler",
            "container_id": "597a3518-88a1-4375-a41b-3498f05c5293"
        },
        {
            "type": "AtomicRuler",
            "container_id": "e0a66af3-6b39-4ec2-8628-522783c50e03"
        },
        {
            "type": "AtomicRuler",
            "container_id": "5ffb7d8a-b0d7-446f-9772-95fb90a2d5cd"
        }
    ],
    "annotations": [
        {
            "type": "Utterance",
            "value": {
                "chat_id": "55cade77-3272-410e-801d-8cc483ae8f16",
                "utterance": "That is my brother Jim\n",
                "seq": [
                    {
                        "value": "That",
                        "id": "7bb5ca79-5b25-4864-a2c9-14f67d

In [8]:
utt_ruler = utterance_annotation.value.ruler

token = tokens[4]
jim = Friend(None, token.value, 32, Gender.MALE)
referent_annotation = Annotation(jim, "annotator_1", time.time())
referent_mention = Mention(utt_ruler.get_offset(4,4), [referent_annotation])
text_signal.mentions.append(referent_mention)

print(json.dumps(referent_mention, default=serializer, indent=4))

{
    "segment": {
        "type": "Index",
        "container_id": "2fe66671-390e-4d08-a89e-d97d12f5283e",
        "start": 4,
        "stop": 4
    },
    "annotations": [
        {
            "type": "Friend",
            "value": {
                "id": "d8277be5-dd36-4792-b795-fc6c929409bc",
                "name": "Jim",
                "age": 32,
                "gender": "MALE"
            },
            "source": "annotator_1",
            "timestamp": 1605002037.04578
        }
    ]
}


In [9]:
triple_segments = (utt_ruler.get_offset(2,2), utt_ruler.get_offset(3,3), utt_ruler.get_offset(4,4))
triple = Triple.from_friends(str(speaker.id), "brother-of", str(jim.id))
triple_annotation = Annotation(triple, "annotator_1", time.time())
triple_mention = Mention(triple_segments, [triple_annotation])
text_signal.mentions.append(triple_mention)

print(json.dumps(triple_mention, default=serializer, indent=4))

{
    "segment": [
        {
            "type": "Index",
            "container_id": "2fe66671-390e-4d08-a89e-d97d12f5283e",
            "start": 2,
            "stop": 2
        },
        {
            "type": "Index",
            "container_id": "2fe66671-390e-4d08-a89e-d97d12f5283e",
            "start": 3,
            "stop": 3
        },
        {
            "type": "Index",
            "container_id": "2fe66671-390e-4d08-a89e-d97d12f5283e",
            "start": 4,
            "stop": 4
        }
    ],
    "annotations": [
        {
            "type": "Triple",
            "value": {
                "subject": {
                    "id": "http://cltl.nl/leolani/friends/dc74258c-2849-4c62-b6b8-e6df13d8e653",
                    "type": "FRIEND"
                },
                "predicate": "http://cltl.nl/combot/predicate/brother-of",
                "object": {
                    "id": "http://cltl.nl/leolani/friends/d8277be5-dd36-4792-b795-fc6c929409bc",
                 

In [10]:
print(json.dumps(text_signal, default=serializer, indent=4))
append_signal(scenario.signals[Modality.TEXT.name.lower()], text_signal)

{
    "modality": "TEXT",
    "time": {
        "type": "TemporalRuler",
        "container_id": "test_scenario",
        "start": 1603139850,
        "end": 1603149890
    },
    "files": [
        "data/test-scenes/test_scenario/text/chat1_utterance1.txt"
    ],
    "mentions": [
        {
            "segment": {
                "type": "Index",
                "container_id": "2f011b41-28ed-4184-8e6c-fcdeac4c964b",
                "start": 0,
                "stop": 4
            },
            "annotations": [
                {
                    "type": "Token",
                    "value": {
                        "value": "That",
                        "id": "7bb5ca79-5b25-4864-a2c9-14f67d1c2287",
                        "ruler": {
                            "type": "AtomicRuler",
                            "container_id": "7bb5ca79-5b25-4864-a2c9-14f67d1c2287"
                        }
                    },
                    "source": "treebank_tokenizer",
            

Define the annotation that goes with a family picture

In [11]:
family_img_file = scenario_data_path(scenario_id, "pexels-victoria-borodinova-1648358.jpg", Modality.IMAGE)
family_img_time = scenario.ruler.get_time_segment(1603139840, 1603149860)

family_img_signal = ImageSignal(None, family_img_time, [family_img_file], ((0, 3456), (0, 5184)))
jim_bounding_box = family_img_signal.ruler.get_area_bounding_box(450,1280,1200,1900)

jim_annotation = Annotation(jim, "face_recognition", time.time())
jim_mention = Mention(jim_bounding_box, [jim_annotation])
family_img_signal.mentions.append(jim_mention)

print(json.dumps(family_img_signal, default=serializer, indent=4))
append_signal(scenario.signals[Modality.IMAGE.name.lower()], family_img_signal)

{
    "modality": "IMAGE",
    "time": {
        "type": "TemporalRuler",
        "container_id": "test_scenario",
        "start": 1603139840,
        "end": 1603149860
    },
    "files": [
        "data/test-scenes/test_scenario/image/pexels-victoria-borodinova-1648358.jpg"
    ],
    "mentions": [
        {
            "segment": {
                "type": "MultiIndex",
                "container_id": "e615a9eb-4019-4f6d-938f-6c50c808719b",
                "bounds": [
                    [
                        450,
                        1200
                    ],
                    [
                        1280,
                        1900
                    ]
                ]
            },
            "annotations": [
                {
                    "type": "Friend",
                    "value": {
                        "id": "d8277be5-dd36-4792-b795-fc6c929409bc",
                        "name": "Jim",
                        "age": 32,
                        "

Annotation of the communication about the picture(s)

In [12]:
for signal_file in scenario.signals.values():
    append_signal(signal_file, None, terminate=True)

### Convert to Leolani domain

In [13]:
# from integration.convert import convert, integrate_image_signal, integrate_text_signal

In [14]:
# leolani_context = convert(scenario)
# print(json.dumps(leolani_context, default=vars, indent=4))

In [15]:
# print(str([(topic, vars(ev)) for topic, ev in integrate_image_signal(speaker_img_signal)]))

In [16]:
# events, triples = integrate_text_signal(text_signal, leolani_context)
# print(str([(topic, vars(ev)) for topic, ev in events]))
# print()
# print(triples)

In [17]:
# print(json.dumps(leolani_context, default=vars, indent=4))